In [1]:
import os
import django
import requests
import datetime
import json
from tqdm import tqdm
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
# https://developers.themoviedb.org/3/discover/movie-discover
URL = 'https://api.themoviedb.org/3'
API_KEY = '164acb58532a315bea423c96031d8a71'

## 장르 DB

In [3]:
url_genres = f'{URL}/genre/movie/list?api_key=164acb58532a315bea423c96031d8a71'

In [4]:
result = requests.get(url_genres)

In [5]:
genres = result.json()['genres']

In [6]:
for g in genres:
    genre = Genre()
    genre.pk = g['id']
    genre.genre_name = g['name']
    genre.save()

## 영화 DB

In [3]:
def date_term(year):
    from_date = [f'{year}-{str(i).zfill(2)}-01' for i in range(1, 13)]
    to_date = [f'{year}-{str(i).zfill(2)}-31' for i in range(1, 13)]
    return from_date, to_date

In [4]:
def extend_movies(params):
    result = requests.get(f'{URL}/discover/movie', params=  params).json()
    movies.extend(result['results'])
    if params['page'] == 1:
        return result['total_pages']

In [ ]:
for year in reversed(range(2015, 2018)):
    movies = []
    from_date, to_date = date_term(year)
    for month in reversed(range(12)):
        print(f'{year}년{month+1}월 수집중')
        params = {
            'api_key': API_KEY,
            'primary_release_date.gte': from_date[month],
            'primary_release_date.lte': to_date[month],
            'language': 'ko',
            'page': 1
        }
        total_pages = extend_movies(params)
        print(f'해당 월의 전체 페이지 수 : {total_pages}')
        for page_num in range(2, total_pages + 1):
            print(page_num, end='-')
            params['page'] = page_num
            extend_movies(params)
    with open(f'./movie_{year}.json', 'w') as fp:
        json.dump(movies, fp)

## Movies to DB

In [7]:
movies[0]

{'adult': False,
 'backdrop_path': '/7d6EY00g1c39SGZOoCJ5Py9nNth.jpg',
 'genre_ids': [28, 12, 16, 878, 35],
 'id': 324857,
 'original_language': 'en',
 'original_title': 'Spider-Man: Into the Spider-Verse',
 'overview': '평범한 10대 마일스 모랄레스는 우연히 방사능 거미에 물려 스파이더맨 능력을 가지게 된다. 혼란스러워하던 마일스는 악당과 싸우고 있는 피터 파커를 마주치게 되고 피터 파커는 마일스가 자신과 같은 능력을 가지고 있음을 직감한다. 여러 개의 평행세계가 존재한다는 것을 알게 된 마일스와 피터 파커는 이후 스파이더우먼 스파이더 그웬, 스파이더맨 누아르, 스파이더햄 등 평행세계 속 공존하는 모든 스파이더맨들을 만나게 되는데... 하나의 유니버스에서 만나 팀을 결성한 스파이더맨들은 과연 세계를 구할 수 있을까?',
 'popularity': 88.136,
 'poster_path': '/vnWgIIEWAvWKOI65tm6h6VRbyY8.jpg',
 'release_date': '2018-12-06',
 'title': '스파이더맨: 뉴 유니버스',
 'video': False,
 'vote_average': 8.4,
 'vote_count': 9415}

In [5]:
with open(f'./movie_{2018}.json', 'r') as fp:
    movies = json.load(fp)

In [18]:
import warnings
warnings.filterwarnings(action='ignore')

In [46]:
# # using bulk
# to_db_movies = []

# for mv in movies[:10]:
#     movie = Movie()
#     movie.title_en = mv['original_title']
#     movie.title_ko = mv['title']
#     movie.rate = mv['vote_average']
#     movie.rate_people_count = mv['vote_count']
#     movie.poster_path = mv['poster_path']
#     movie.description = mv['overview']
#     movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
#     to_db_movies.append(movie)

# Movie.objects.bulk_create(to_db_movies)

In [11]:
with open(f'./movie_{2021}.json', 'r') as fp:
    movies = json.load(fp)

In [19]:
for mv in tqdm(movies[10:]):
    movie = Movie()
    movie.pk = mv['id']
    movie.title_en = mv['original_title']
    movie.title_ko = mv['title']
    movie.rate = mv['vote_average']
    movie.rate_people_count = mv['vote_count']
    movie.poster_path = mv['poster_path']
    movie.description = mv['overview']
    movie.release_date = datetime.datetime.strptime(mv['release_date'], '%Y-%m-%d')
    movie.save()
    for gid in mv['genre_ids']:
        movie.genres.add(Genre.objects.get(pk=gid))

100%|██████████████████████████████████████████████████████████████████████████████████████| 8793/8793 [02:12<00:00, 66.14it/s]
